# Klasa TCM

In [1]:
from pymprog import begin, var, maximize, solve, end, iprod, par, minimize, vobj, solver, glpk, sensitivity

from numpy import array, zeros, ones, amax, transpose, reshape, size, eye,\
    concatenate, shape, sum, append
from scipy.optimize._linprog import linprog
from numpy.random import permutation
from itertools import permutations
from time import clock
import xlwt
from pylab import plot, show, bar
from math import inf

class TCM(object):
    
    def __init__(self,t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb):
        '''t-tablica czasow trwania
        tco - sprzezenia czasowe pomiedzy obiektami
        tcb - sprzezenia czasowe pomiedzy brygadami
        wco - wagi sprzezen pomiedzy obiektami
        wcb - wagi sprzezn pomiedzy brygadami
        '''
                
        self.ro=0.0001
        
        self.t=t
        
        self.tcdo=tcdo
        self.tcdb=tcdb
        
        self.tcgo=tcgo
        self.tcgb=tcgb
        
        self.cwdo=cwdo
        self.cwdb=cwdb
        
        self.cwgo=cwgo
        self.cwgb=cwgb
        
        (self.onumber,self.bnumber)=shape(self.t)
        
        self.solve()

            

        
    def solve(self):
        
        p=begin('TCM')
        W=iprod(range(self.onumber),range(self.bnumber)) # tablica zamiennych decyzyjnych dla NWR NWZ NPR NPZ
        O=iprod(range(self.onumber-1),range(self.bnumber)) #tablica zmiennych dla cko - sprzezen korygujacych pomiedzy obiektami
        B=iprod(range(self.onumber),range(self.bnumber-1)) #tablica zmiennych dla ckb - sprzezen korygujacych pomiedzy brygadami
        
        NWR=var('NWR',W) #czas trwania czynnosci
        NWZ=var('NWZ',W)
        NPR=var('NPR',W)
        NPZ=var('NPZ',W)
        ckdo=var('ckdo',O)
        ckdb=var('ckdb',B)
        ckgo=var('ckgo',O)
        ckgb=var('ckgb',B)
        ZC=var('ZC',W)
        
        t=par('t',self.t)
        
        tcdo=par('tcdo',self.tcdo) #parametry
        tcdb=par('tcdb',self.tcdb)
        
        tcgo=par('tcgo',self.tcgo)
        tcgb=par('tcgb',self.tcgb)
        
        cwdo=par('cwdo',self.cwdo)
        cwdb=par('cwdb',self.cwdb)
        
        cwgo=par('cwgo',self.cwgo)
        cwgb=par('cwgb',self.cwgb)
        

        #funkcja celu - minimalne koszty
        minimize(sum([ckdo[i,j]*cwdo[i][j]+ckgo[i,j]*cwgo[i][j] for (i,j) in O])+sum([ckdb[i,j]*cwdb[i][j]+ckgb[i,j]*cwgb[i][j] for (i,j) in B])+sum([NWZ[i,j]*self.ro-NPZ[i,j]*self.ro for (i,j) in W])+NWZ[self.onumber-1,self.bnumber-1])
        
        NWR[0,0] == 0
        
        for (i,j) in W:
            
            NWZ[i,j]==NWR[i,j]+t[i][j]
            NPZ[i,j]==NPR[i,j]+t[i][j]
            ZC[i,j]==NPZ[i,j]-NWZ[i,j]
            
        for (i,j) in O:
            NWR[i+1,j]>=NWZ[i,j]+tcdo[i][j]-ckdo[i,j]
            NPR[i+1,j]>=NPZ[i,j]+tcdo[i][j]-ckdo[i,j]
            
            NWR[i+1,j]<=NWZ[i,j]+tcgo[i][j]+ckgo[i,j]
            NPR[i+1,j]<=NPZ[i,j]+tcgo[i][j]+ckgo[i,j]
            
        for (i,j) in B:
            NWR[i,j+1]>=NWZ[i,j]+tcdb[i][j]-ckdb[i,j]
            NPR[i,j+1]>=NPZ[i,j]+tcdb[i][j]-ckdb[i,j]
            
            NWR[i,j+1]<=NWZ[i,j]+tcgb[i][j]+ckgb[i,j]
            NPR[i,j+1]<=NPZ[i,j]+tcgb[i][j]+ckgb[i,j]
            
        NWZ[self.onumber-1,self.bnumber-1]==NPZ[self.onumber-1,self.bnumber-1]
            
        
        #parametry solvera
        #solver(float,msg_lev=glpk.GLP_MSG_OFF)
        
        p.solve()
        
        print(vobj())
        
        for (i,j) in W:
            print(i,j,'NWR',NWR[i,j].primal,'NWZ',NWZ[i,j].primal,'NPR',NPR[i,j].primal,'NPZ',NPZ[i,j].primal,'ZC',ZC[i,j].primal)
        for (i,j) in O:
            print(i,j,'ckdo',ckdo[i,j].primal,'ckgo',ckgo[i,j].primal)       
        for (i,j) in B:
            print(i,j,'ckdb',ckdb[i,j].primal,'ckgb',ckgb[i,j].primal)
        
        #sensitivity()

# Model 1: Brak ograniczeń

In [2]:
t=[[3,8,5,6],
    [8,9,7,9],
    [4,7,8,7]]
    
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

42.9983
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 11.0 NWZ 16.0 NPR 15.0 NPZ 20.0 ZC 4.0
0 3 NWR 16.0 NWZ 22.0 NPR 21.0 NPZ 27.0 ZC 5.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 11.0 NWZ 20.0 NPR 11.0 NPZ 20.0 ZC 0.0
1 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
1 3 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
2 0 NWR 11.0 NWZ 15.0 NPR 17.0 NPZ 21.0 ZC 6.0
2 1 NWR 20.0 NWZ 27.0 NPR 21.0 NPZ 28.0 ZC 1.0
2 2 NWR 27.0 NWZ 35.0 NPR 28.0 NPZ 36.0 ZC 1.0
2 3 NWR 36.0 NWZ 43.0 NPR 36.0 NPZ 43.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 2: Ciagłość pracy brygad

In [3]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[0,0,0,0],
    [0,0,0,0]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

43.0
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 15.0 NWZ 20.0 NPR 15.0 NPZ 20.0 ZC 0.0
0 3 NWR 21.0 NWZ 27.0 NPR 21.0 NPZ 27.0 ZC 0.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 11.0 NWZ 20.0 NPR 11.0 NPZ 20.0 ZC 0.0
1 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
1 3 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
2 0 NWR 11.0 NWZ 15.0 NPR 11.0 NPZ 15.0 ZC 0.0
2 1 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
2 2 NWR 27.0 NWZ 35.0 NPR 27.0 NPZ 35.0 ZC 0.0
2 3 NWR 36.0 NWZ 43.0 NPR 36.0 NPZ 43.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 3: Ciagłość pracy na obiektach

In [4]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

42.99999999999999
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 11.0 NWZ 16.0 NPR 11.0 NPZ 16.0 ZC 0.0
0 3 NWR 16.0 NWZ 22.0 NPR 16.0 NPZ 22.0 ZC 0.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 11.0 NWZ 20.0 NPR 11.0 NPZ 20.0 ZC 0.0
1 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
1 3 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
2 0 NWR 17.0 NWZ 21.0 NPR 17.0 NPZ 21.0 ZC 0.0
2 1 NWR 21.0 NWZ 28.0 NPR 21.0 NPZ 28.0 ZC 0.0
2 2 NWR 28.0 NWZ 36.0 NPR 28.0 NPZ 36.0 ZC 0.0
2 3 NWR 36.0 NWZ 43.0 NPR 36.0 NPZ 43.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 4: Ciagłość pracy dla 3 obiektu i 3 brygady

In [5]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,0,100],
    [100,100,0,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [0,0,0]]
    
cwdo=[[100,100,10000,100],
    [100,100,10000,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [10000,10000,10000]]
    
cwgo=[[100,100,10000,100],
    [100,100,10000,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [10000,10000,10000]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

141.9999
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 15.0 NWZ 20.0 NPR 15.0 NPZ 20.0 ZC 0.0
0 3 NWR 20.0 NWZ 26.0 NPR 21.0 NPZ 27.0 ZC 1.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 11.0 NWZ 20.0 NPR 11.0 NPZ 20.0 ZC 0.0
1 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
1 3 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
2 0 NWR 16.0 NWZ 20.0 NPR 16.0 NPZ 20.0 ZC 0.0
2 1 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
2 2 NWR 27.0 NWZ 35.0 NPR 27.0 NPZ 35.0 ZC 0.0
2 3 NWR 35.0 NWZ 42.0 NPR 35.0 NPZ 42.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 1.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 5: Jedna brygada może pracować na kilku obiektach

In [6]:
tcdo=[[-1,-1,-1,-1],
    [-1,-1,-1,-1]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

40.99829999999999
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 11.0 NWZ 16.0 NPR 15.0 NPZ 20.0 ZC 4.0
0 3 NWR 16.0 NWZ 22.0 NPR 21.0 NPZ 27.0 ZC 5.0
1 0 NWR 2.0 NWZ 10.0 NPR 2.0 NPZ 10.0 ZC 0.0
1 1 NWR 10.0 NWZ 19.0 NPR 10.0 NPZ 19.0 ZC 0.0
1 2 NWR 19.0 NWZ 26.0 NPR 19.0 NPZ 26.0 ZC 0.0
1 3 NWR 26.0 NWZ 35.0 NPR 26.0 NPZ 35.0 ZC 0.0
2 0 NWR 9.0 NWZ 13.0 NPR 15.0 NPZ 19.0 ZC 6.0
2 1 NWR 18.0 NWZ 25.0 NPR 19.0 NPZ 26.0 ZC 1.0
2 2 NWR 25.0 NWZ 33.0 NPR 26.0 NPZ 34.0 ZC 1.0
2 3 NWR 34.0 NWZ 41.0 NPR 34.0 NPZ 41.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 6: Na jednym obiekcie może pracować więcej niż jedna brygada

In [7]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[-1,-1,-1],
    [-1,-1,-1],
    [-1,-1,-1]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

39.9983
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 2.0 NWZ 10.0 NPR 2.0 NPZ 10.0 ZC 0.0
0 2 NWR 9.0 NWZ 14.0 NPR 13.0 NPZ 18.0 ZC 4.0
0 3 NWR 13.0 NWZ 19.0 NPR 18.0 NPZ 24.0 ZC 5.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 10.0 NWZ 19.0 NPR 10.0 NPZ 19.0 ZC 0.0
1 2 NWR 18.0 NWZ 25.0 NPR 18.0 NPZ 25.0 ZC 0.0
1 3 NWR 24.0 NWZ 33.0 NPR 24.0 NPZ 33.0 ZC 0.0
2 0 NWR 11.0 NWZ 15.0 NPR 17.0 NPZ 21.0 ZC 6.0
2 1 NWR 19.0 NWZ 26.0 NPR 20.0 NPZ 27.0 ZC 1.0
2 2 NWR 25.0 NWZ 33.0 NPR 26.0 NPZ 34.0 ZC 1.0
2 3 NWR 33.0 NWZ 40.0 NPR 33.0 NPZ 40.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 7: Zazębianie się pomiędzy obiektami i brygadami

In [8]:
tcdo=[[-1,-1,-1,-1],
    [-1,-1,-1,-1]]
tcdb=[[-1,-1,-1],
    [-1,-1,-1],
    [-1,-1,-1]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

37.99829999999999
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 2.0 NWZ 10.0 NPR 2.0 NPZ 10.0 ZC 0.0
0 2 NWR 9.0 NWZ 14.0 NPR 13.0 NPZ 18.0 ZC 4.0
0 3 NWR 13.0 NWZ 19.0 NPR 18.0 NPZ 24.0 ZC 5.0
1 0 NWR 2.0 NWZ 10.0 NPR 2.0 NPZ 10.0 ZC 0.0
1 1 NWR 9.0 NWZ 18.0 NPR 9.0 NPZ 18.0 ZC 0.0
1 2 NWR 17.0 NWZ 24.0 NPR 17.0 NPZ 24.0 ZC 0.0
1 3 NWR 23.0 NWZ 32.0 NPR 23.0 NPZ 32.0 ZC 0.0
2 0 NWR 9.0 NWZ 13.0 NPR 15.0 NPZ 19.0 ZC 6.0
2 1 NWR 17.0 NWZ 24.0 NPR 18.0 NPZ 25.0 ZC 1.0
2 2 NWR 23.0 NWZ 31.0 NPR 24.0 NPZ 32.0 ZC 1.0
2 3 NWR 31.0 NWZ 38.0 NPR 31.0 NPZ 38.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0


# Model 8: Harmonogramowanie priorytetowe

In [9]:
t=[[3,8,5,6],
    [8,9,7,9],
    [4,7,8,7]]
    
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,0,0,100],
    [100,0,0,100]]
tcgb=[[100,100,100],
    [0,0,0],
    [100,100,100]]
    
cwdo=[[100,10000,100000000,100],
    [100,10000,100000000,100]]
cwdb=[[100,100,100],
    [1000000,1000000,1000000],
    [100,100,100]]
    
cwgo=[[100,10000,100000000,100],
    [100,10000,100000000,100]]
cwgb=[[100,100,100],
    [1000000,1000000,1000000],
    [100,100,100]]


TCM(t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

42.999399999999994
0 0 NWR 0.0 NWZ 3.0 NPR 0.0 NPZ 3.0 ZC 0.0
0 1 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
0 2 NWR 15.0 NWZ 20.0 NPR 15.0 NPZ 20.0 ZC 0.0
0 3 NWR 20.0 NWZ 26.0 NPR 21.0 NPZ 27.0 ZC 1.0
1 0 NWR 3.0 NWZ 11.0 NPR 3.0 NPZ 11.0 ZC 0.0
1 1 NWR 11.0 NWZ 20.0 NPR 11.0 NPZ 20.0 ZC 0.0
1 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
1 3 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
2 0 NWR 11.0 NWZ 15.0 NPR 16.0 NPZ 20.0 ZC 5.0
2 1 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
2 2 NWR 27.0 NWZ 35.0 NPR 27.0 NPZ 35.0 ZC 0.0
2 3 NWR 36.0 NWZ 43.0 NPR 36.0 NPZ 43.0 ZC 0.0
0 0 ckdo 0.0 ckgo 0.0
0 1 ckdo 0.0 ckgo 0.0
0 2 ckdo 0.0 ckgo 0.0
0 3 ckdo 0.0 ckgo 0.0
1 0 ckdo 0.0 ckgo 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
0 0 ckdb 0.0 ckgb 0.0
0 1 ckdb 0.0 ckgb 0.0
0 2 ckdb 0.0 ckgb 0.0
1 0 ckdb 0.0 ckgb 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
2 0 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
